# requirements.txt

In [5]:
!pip install ultralytics
!pip install torch==1.13.0+cu116 torchvision==0.14.0+cu116 torchaudio==0.13.0 --extra-index-url https://download.pytorch.org/whl/cu116
!pip install numpy
!pip install pandas
!pip install Pillow
!pip install pyYAML
!pip install requests
!pip install tqdm
!pip install sahi

Looking in indexes: https://pypi.org/simple, https://download.pytorch.org/whl/cu116
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.0/104.0 kB 3.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.3/88.3 kB 6.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Using cached click-8.1.3-py3-none-any.whl (96 kB)
  Using cached terminaltables-3.1.10-py2.py3-none-any.whl (15 kB)
  Using cached shapely-2.0.1-cp38-cp38-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (2.3 MB)
  Using cached termcolor-2.3.0-py3-none-any.whl (6.9 kB)
  Created wheel for fire: filename=fire-0.5.0-py2.py3-none-any.whl size=116931 sha256=c9b3813fb3020ca401407c150f6105ac4c98ce270899f7d4ac206d8bfffa5a90
  Stored in directory: /home/int2/.cache/pip/wheels/5b/eb/43/7295e71293b218ddfd627f935229bf54af9018add7fbb5aac6
Successfully built fire


# convert coco to yolo

In [46]:
import json
import os

In [28]:
with open("../datasets/sample_dataset/train/_annotations.coco.json", "r") as train_json:
    train_dataset = json.load(train_json)
    train_json.close()

with open("../datasets/sample_dataset/valid/_annotations.coco.json", "r") as valid_json:
    valid_dataset = json.load(valid_json)
    valid_json.close()

with open("../datasets/sample_dataset/test/_annotations.coco.json", "r") as test_json:
    test_dataset = json.load(test_json)
    test_json.close()

In [29]:
print(train_dataset.keys())
print(valid_dataset.keys())
print(test_dataset.keys())

dict_keys(['info', 'licenses', 'categories', 'images', 'annotations'])
dict_keys(['info', 'licenses', 'categories', 'images', 'annotations'])
dict_keys(['info', 'licenses', 'categories', 'images', 'annotations'])


In [30]:
test_dataset["categories"]

[{'id': 0, 'name': 'symptom', 'supercategory': 'none'},
 {'id': 1, 'name': 'json', 'supercategory': 'symptom'},
 {'id': 2, 'name': 'object', 'supercategory': 'symptom'}]

In [31]:
test_dataset["images"]

[{'id': 0,
  'license': 1,
  'file_name': 'F01_U01_O0092_D2022-07-04_L370_W0573_S3_R01_B02_I00071719_JPG.rf.44c434afe2262358ae5bf76c327fd3e0.jpg',
  'height': 640,
  'width': 640,
  'date_captured': '2023-05-30T01:37:03+00:00'},
 {'id': 1,
  'license': 1,
  'file_name': 'F01_U01_O0089_D2022-07-04_L335_W0379_S2_R02_B03_I00000007_JPG.rf.c784de0d8fff0c25bbc4c3f274ef8dab.jpg',
  'height': 640,
  'width': 640,
  'date_captured': '2023-05-30T01:37:03+00:00'},
 {'id': 2,
  'license': 1,
  'file_name': 'F01_U01_O0089_D2022-07-04_L335_W0379_S2_R05_B02_I00000007_JPG.rf.72c1d649bc84d88f105a19e985d417ce.jpg',
  'height': 640,
  'width': 640,
  'date_captured': '2023-05-30T01:37:03+00:00'},
 {'id': 3,
  'license': 1,
  'file_name': 'F01_U01_O0092_D2022-07-04_L370_W0573_S3_R02_B03_I00071719_JPG.rf.f8946c8b593ee66ba87d5cc89fa5da57.jpg',
  'height': 640,
  'width': 640,
  'date_captured': '2023-05-30T01:37:03+00:00'},
 {'id': 4,
  'license': 1,
  'file_name': 'F01_U01_O0090_D2022-07-04_L325_W0295_S2_R

In [32]:
test_dataset["annotations"]

[{'id': 0,
  'image_id': 0,
  'category_id': 2,
  'bbox': [101, 260, 97, 122.5],
  'area': 11882.5,
  'segmentation': [],
  'iscrowd': 0},
 {'id': 1,
  'image_id': 0,
  'category_id': 2,
  'bbox': [284, 404, 91, 36.5],
  'area': 3321.5,
  'segmentation': [],
  'iscrowd': 0},
 {'id': 2,
  'image_id': 0,
  'category_id': 2,
  'bbox': [510, 320, 28.5, 59.5],
  'area': 1695.75,
  'segmentation': [],
  'iscrowd': 0},
 {'id': 3,
  'image_id': 1,
  'category_id': 2,
  'bbox': [182, 211, 13, 10.5],
  'area': 136.5,
  'segmentation': [],
  'iscrowd': 0},
 {'id': 4,
  'image_id': 1,
  'category_id': 2,
  'bbox': [449, 380, 59.5, 115.5],
  'area': 6872.25,
  'segmentation': [],
  'iscrowd': 0},
 {'id': 5,
  'image_id': 1,
  'category_id': 2,
  'bbox': [176, 167, 332.5, 328.5],
  'area': 109226.25,
  'segmentation': [],
  'iscrowd': 0},
 {'id': 6,
  'image_id': 2,
  'category_id': 2,
  'bbox': [352, 205, 63.5, 36.5],
  'area': 2317.75,
  'segmentation': [],
  'iscrowd': 0},
 {'id': 7,
  'image_id'

In [43]:
[category["name"] for category in test_dataset["categories"]]

['symptom', 'json', 'object']

In [50]:
def json2yolo(dataset, save_path):
    os.makedirs(save_path, exist_ok=True)

    categories = [category["name"] for category in dataset["categories"]]
    save_categories_name = os.path.join(save_path, "classes.txt")
    lines = ""
    for category in categories:
        lines += f"{category}\n"
    
    with open(save_categories_name, "w") as f:
        f.writelines(lines)
        f.close()

    for img, anno in zip(dataset["images"], dataset["annotations"]):
        lines = ""
        file_name = img["file_name"]
        save_name = os.path.join(save_path, file_name).replace(".jpg", ".txt")
        height = img["height"]
        width = img["width"]
        category = anno["category_id"]
        bbox = anno["bbox"]
        x1, y1, x2, y2 = bbox[0], bbox[1], bbox[0]+bbox[2], bbox[1]+bbox[3]
        
        x = (x1+x2)/2/width
        y = (y1+y2)/2/height
        w = (x2-x1)/width
        h = (y2-y1)/height
        
        lines += f"{category} {round(x,6)} {round(y,6)} {round(w,6)} {round(h,6)}\n"
        
        with open(save_name, "w") as f:
            f.writelines(lines)
            f.close()

In [51]:
json2yolo(train_dataset, "../datasets/sample_dataset/train/labels")
json2yolo(valid_dataset, "../datasets/sample_dataset/valid/labels")
json2yolo(test_dataset, "../datasets/sample_dataset/test/labels")